Основная часть

Задание 1: Простая линейная регрессия.  
Используйте набор данных "Boston Housing" из sklearn.datasets. Постройте модель линейной регрессии, сделайте предсказания и вычислите MSE (Mean Squared Error).

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

data = fetch_california_housing(as_frame=True)
X = data.data  # Признаки
y = data.target  # Целевая переменная (стоимость жилья)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()

model.fit(X_train, y_train)

# Предсказания на тестовой выборке
y_pred = model.predict(X_test)

# MSE
mse = mean_squared_error(y_test, y_pred)

print(f"Среднеквадратичная ошибка (MSE): {mse}")


Среднеквадратичная ошибка (MSE): 0.555891598695244


Задание 2: Использование кросс-валидации.  
С использованием того же набора данных проведите k-fold кросс-валидацию (k=10) для своей модели и сравните среднее значения MSE на всех фолдах.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

data = fetch_california_housing(as_frame=True)
X = data.data
y = data.target

# Он делит весь набор данных на несколько частей (фолдов), 
# чтобы модель обучалась на разных подвыборках данных, а 
# затем тестировалась на оставшихся.
kf = KFold(n_splits=10, shuffle=True, random_state=42)
mse_values = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = LinearRegression()    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mse_values.append(mse)

mean_mse = np.mean(mse_values)

print(f"MSE на каждом фолде: {mse_values}")
print(f"Среднее значение MSE: {mean_mse}")


MSE на каждом фолде: [0.5590019245025633, 0.5531232970331572, 0.4811612796368561, 0.5742036665176363, 0.4889758378191175, 0.5283881757036769, 0.5511782380281002, 0.47483961298015503, 0.5643692207197094, 0.5500253663196297]
Среднее значение MSE: 0.5325266619260601


Задание 3: Построение Ridge регрессии.  
Примените Ridge регрессию к набору данных "Boston Housing". Подберите гиперпараметр `alpha` через кросс-валидацию. 

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

data = fetch_california_housing(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение модели Ridge-регрессии
ridge = Ridge()

# Подбор гиперпараметра alpha через кросс-валидацию
param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}  # Возможные значения alpha
grid_search = GridSearchCV(ridge, param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Оптимальное значение alpha
best_alpha = grid_search.best_params_['alpha']
print(f"Оптимальное значение alpha: {best_alpha}")

# Оценка модели с оптимальным alpha
best_ridge = Ridge(alpha=best_alpha)
best_ridge.fit(X_train, y_train)

# Предсказания и вычисление MSE
y_pred = best_ridge.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Среднеквадратичная ошибка (MSE): {mse}")


Оптимальное значение alpha: 10
Среднеквадратичная ошибка (MSE): 0.5550405537343013


Задание 4: Построение Lasso регрессии.  
Также примените Lasso регрессию к тем же данным. При подборе гиперпараметра `alpha` через кросс-валидацию сравните количество нулевых весов в модели с результатами Ridge регрессии.

In [6]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import numpy as np

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определение модели Lasso-регрессии
lasso = Lasso(max_iter=10000)

# Подбор гиперпараметра alpha через кросс-валидацию
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}  # Возможные значения alpha
grid_search = GridSearchCV(lasso, param_grid, cv=10, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Оптимальное значение alpha
best_alpha = grid_search.best_params_['alpha']
print(f"Оптимальное значение alpha для Lasso: {best_alpha}")

# Оценка модели с оптимальным alpha
best_lasso = Lasso(alpha=best_alpha, max_iter=10000)
best_lasso.fit(X_train, y_train)

# Предсказания и вычисление MSE
y_pred = best_lasso.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Среднеквадратичная ошибка (MSE) для Lasso: {mse}")

# Сравнение количества нулевых весов
ridge_coefs = best_ridge.coef_  # Коэффициенты из Ridge-регрессии
lasso_coefs = best_lasso.coef_  # Коэффициенты из Lasso-регрессии

ridge_zero_weights = np.sum(ridge_coefs == 0)  # Количество нулевых коэффициентов в Ridge
lasso_zero_weights = np.sum(lasso_coefs == 0)  # Количество нулевых коэффициентов в Lasso

print(f"Количество нулевых коэффициентов в Ridge: {ridge_zero_weights}")
print(f"Количество нулевых коэффициентов в Lasso: {lasso_zero_weights}")

Оптимальное значение alpha для Lasso: 0.001
Среднеквадратичная ошибка (MSE) для Lasso: 0.5538940157172418
Количество нулевых коэффициентов в Ridge: 0
Количество нулевых коэффициентов в Lasso: 0


Задание 5: Использование других функций потерь.  
Используйте набор данных diabetes из sklearn.datasets и постройте модель HuberRegressor - линейную модель с функцией потерь Хьюбера, которая менее чувствительна к выбросам по сравнению с MSE.


In [10]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import HuberRegressor
from sklearn.metrics import mean_squared_error

data = load_diabetes(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

huber = HuberRegressor(max_iter=1000)

huber.fit(X_train, y_train)

y_pred = huber.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Среднеквадратичная ошибка (MSE): {mse}")


Среднеквадратичная ошибка (MSE): 2913.7072165243917


Сравнение моделей

Задание 6: Исследование эффекта масштабирования признаков.  
Используйте любой набор данных с числовыми признаками. Тренируйте модели Ridge и Lasso регрессии на исходных данных и предобработанных данных (используйте стандартизацию и нормализацию). Сравните коэффициенты моделей, полученных для исходных и предобработанных данных.

In [12]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd

data = fetch_california_housing(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание моделей Ridge и Lasso
ridge = Ridge(alpha=1.0)
lasso = Lasso(alpha=0.1, max_iter=10000)

# Обучение моделей на исходных данных
ridge.fit(X_train, y_train)
lasso.fit(X_train, y_train)

# Коэффициенты для исходных данных
ridge_coefs_original = ridge.coef_
lasso_coefs_original = lasso.coef_

# Стандартизация
scaler_standard = StandardScaler()
X_train_standard = scaler_standard.fit_transform(X_train)
X_test_standard = scaler_standard.transform(X_test)

# Нормализация
scaler_minmax = MinMaxScaler()
X_train_minmax = scaler_minmax.fit_transform(X_train)
X_test_minmax = scaler_minmax.transform(X_test)

# Обучение моделей на стандартизированных данных
ridge.fit(X_train_standard, y_train)
lasso.fit(X_train_standard, y_train)

# Коэффициенты для стандартизированных данных
ridge_coefs_standard = ridge.coef_
lasso_coefs_standard = lasso.coef_

# Обучение моделей на нормализованных данных
ridge.fit(X_train_minmax, y_train)
lasso.fit(X_train_minmax, y_train)

# Коэффициенты для нормализованных данных
ridge_coefs_minmax = ridge.coef_
lasso_coefs_minmax = lasso.coef_

# Сравнение коэффициентов
coefs_comparison = pd.DataFrame({
    'Feature': data.feature_names,
    'Ridge_Original': ridge_coefs_original,
    'Ridge_Standard': ridge_coefs_standard,
    'Ridge_MinMax': ridge_coefs_minmax,
    'Lasso_Original': lasso_coefs_original,
    'Lasso_Standard': lasso_coefs_standard,
    'Lasso_MinMax': lasso_coefs_minmax
})

print("Сравнение коэффициентов моделей Ridge и Lasso:")
print(coefs_comparison.to_string(index=False))

Сравнение коэффициентов моделей Ridge и Lasso:
   Feature  Ridge_Original  Ridge_Standard  Ridge_MinMax  Lasso_Original  Lasso_Standard  Lasso_MinMax
    MedInc        0.448511        0.854327      5.704374        0.392693        0.710598      0.282117
  HouseAge        0.009726        0.122624      0.505032        0.015081        0.106453      0.000000
  AveRooms       -0.123014       -0.294210     -3.066356       -0.000000       -0.000000      0.000000
 AveBedrms        0.781417        0.339008      6.456633        0.000000        0.000000     -0.000000
Population       -0.000002       -0.002282     -0.080924        0.000016       -0.000000     -0.000000
  AveOccup       -0.003526       -0.040833     -2.430280       -0.003149       -0.000000     -0.000000
  Latitude       -0.419787       -0.896168     -4.165575       -0.114291       -0.011469     -0.000000
 Longitude       -0.433681       -0.869071     -4.531451       -0.099308       -0.000000     -0.000000


Задание 7: Изучение влияния регуляризации.  
Выберите набор данных с высокой размерностью признаков. Создайте модели Lasso и Ridge регрессии. Проведите эксперименты с различными степенями регуляризации и установите, как они влияют на производительность модели и распределение весов признаков.

In [15]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Создание набора данных с высокой размерностью
X, y = make_regression(n_samples=500, n_features=100, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создание моделей Ridge и Lasso с различными alpha
alphas = [0.01, 0.1, 1, 10, 100]
results = []

for alpha in alphas:
    # Ridge-регрессия
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    ridge_mse = mean_squared_error(y_test, ridge.predict(X_test_scaled))
    
    # Lasso-регрессия
    lasso = Lasso(alpha=alpha, max_iter=10000)
    lasso.fit(X_train_scaled, y_train)
    lasso_mse = mean_squared_error(y_test, lasso.predict(X_test_scaled))
    
    # Сохранение результатов
    results.append({
        'Alpha': alpha,
        'Ridge_MSE': ridge_mse,
        'Lasso_MSE': lasso_mse,
        'Ridge_Nonzero_Coeffs': np.sum(ridge.coef_ != 0),
        'Lasso_Nonzero_Coeffs': np.sum(lasso.coef_ != 0)
    })

# Создание таблицы с результатами
results_df = pd.DataFrame(results)

# Вывод таблицы результатов
print("Результаты экспериментов с регуляризацией:")
print(results_df.to_string(index=False))

# Alpha - параметр регуляции. Малые (например 0.1) близки к
# обычной линейной регрессии, а большие (например 100.0),
# училивают регуляцию

# Ridge_MSE - среднеквадратичная ошибка на тестовых данных

# Lasso_MSE - среднеквадратичная ошибка на тестовых данных

# Ridge_Nonzero_Coeffs - количество ненулевых коэффициентов.
# В Ridge все коэффициенты остаются ненулевыми, так как 
# L2-регуляризация просто уменьшает их значения, но не обнуляет.

# Lasso_Nonzero_Coeffs - количество ненулевых коэффициентов.
# При малых alpha многие коэффициенты остаются ненулевыми,
# но при высоких alpha L1-регуляризация обнуляет большинство 
# коэффициентов, оставляя только самые важные признаки.

Результаты экспериментов с регуляризацией:
 Alpha   Ridge_MSE    Lasso_MSE  Ridge_Nonzero_Coeffs  Lasso_Nonzero_Coeffs
  0.01    0.012324     0.010596                   100                    13
  0.10    0.015885     0.130142                   100                    10
  1.00    0.379046    10.194521                   100                     8
 10.00   33.973784   797.072090                   100                     6
100.00 1828.130096 31537.044131                   100                     0


Задание 8: Регрессия с использованием метода эластичной сети.  
На том же наборе данных обучите ElasticNet, который объединяет L1 и L2 регуляризацию. Экспериментируйте с разными соотношениями L1 и L2 регуляризации и установите, как это влияет на производительность модели.

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# ElasticNet с различными alpha и l1_ratio
alphas = [0.01, 0.1, 1, 10]
l1_ratios = [0.2, 0.5, 0.8]

results = []

for alpha in alphas:
    for l1_ratio in l1_ratios:
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, max_iter=10000, random_state=42)
        model.fit(X_train_scaled, y_train)
        
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        nonzero_coefs = np.sum(model.coef_ != 0)
        
        results.append({
            'Alpha': alpha,
            'L1_Ratio': l1_ratio,
            'MSE': mse,
            'Nonzero_Coeffs': nonzero_coefs
        })

# Создание таблицы с результатами
results_df = pd.DataFrame(results)

# Вывод таблицы результатов
print("Результаты экспериментов с ElasticNet:")
print(results_df.to_string(index=False))

# Alpha - параметр, регулирующий общую степень регуляризации.
# При малых alpha модель более гибках, но склонна к переобучению
# При больших alpha модель становится устойчивой к переобучению,
# но может терять важные признаки

# L1_Ratio
# L1_Ratio=0.2: Основной акцент на L2-регуляризации (уменьшение
# коэффициентов без обнуления).
# L1_Ratio=0.8: Основной акцент на L1-регуляризации (большая 
# часть коэффициентов обнуляется).

Результаты экспериментов с ElasticNet:
 Alpha  L1_Ratio          MSE  Nonzero_Coeffs
  0.01       0.2     3.638783              99
  0.01       0.5     1.374089              92
  0.01       0.8     0.202768              66
  0.10       0.2   287.100299              99
  0.10       0.5   117.978598              90
  0.10       0.8    18.193626              67
  1.00       0.2  7360.107367              97
  1.00       0.5  4364.244381              87
  1.00       0.8  1151.790695              57
 10.00       0.2 25182.465788              81
 10.00       0.5 22510.909484              43
 10.00       0.8 15464.290292              19


Задание 9: Комбинирование методов отбора признаков и регуляризации.  
Выберите подмножество признаков с помощью любого метода отбора признаков, а затем обучите модели с Lasso и Ridge регуляризацией. Сравнивай модели между собой и с моделью, построенной на всех признаках.

In [19]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

# Отбор подмножества признаков (например, 50 лучших признаков)
k = 50  # Количество лучших признаков
selector = SelectKBest(score_func=f_regression, k=k)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Ridge и Lasso на всех признаках
ridge_all = Ridge(alpha=1.0).fit(X_train_scaled, y_train)
lasso_all = Lasso(alpha=0.1, max_iter=10000).fit(X_train_scaled, y_train)

# Ridge и Lasso на выбранных признаках
ridge_selected = Ridge(alpha=1.0).fit(X_train_selected, y_train)
lasso_selected = Lasso(alpha=0.1, max_iter=10000).fit(X_train_selected, y_train)

# Оценка моделей
results = {
    "Model": ["Ridge (All)", "Lasso (All)", "Ridge (Selected)", "Lasso (Selected)"],
    "MSE": [
        mean_squared_error(y_test, ridge_all.predict(X_test_scaled)),
        mean_squared_error(y_test, lasso_all.predict(X_test_scaled)),
        mean_squared_error(y_test, ridge_selected.predict(X_test_selected)),
        mean_squared_error(y_test, lasso_selected.predict(X_test_selected)),
    ],
    "Nonzero Coefficients": [
        np.sum(ridge_all.coef_ != 0),
        np.sum(lasso_all.coef_ != 0),
        np.sum(ridge_selected.coef_ != 0),
        np.sum(lasso_selected.coef_ != 0),
    ]
}

# Создание таблицы с результатами
results_df = pd.DataFrame(results)
print(results_df)


              Model        MSE  Nonzero Coefficients
0       Ridge (All)   0.379046                   100
1       Lasso (All)   0.130142                    10
2  Ridge (Selected)  98.705961                    50
3  Lasso (Selected)  93.394648                    42


Задание 10: Использование метрик, устойчивых к выбросам.  
Используйте набор данных diabetes из sklearn. и обучите модель HuberRegressor, которая менее чувствительна к выбросам по сравнению с MSE-метрикой. Сравните эту модель с базовой моделью линейной регрессии.


In [23]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

data = load_diabetes(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train)
y_pred_linear = linear_model.predict(X_test_scaled)

# Huber Regressor
huber_model = HuberRegressor(max_iter=1000)
huber_model.fit(X_train_scaled, y_train)
y_pred_huber = huber_model.predict(X_test_scaled)

# Оценка моделей
metrics = {
    "Model": ["Linear Regression", "Huber Regressor"],
    "MSE": [
        mean_squared_error(y_test, y_pred_linear),
        mean_squared_error(y_test, y_pred_huber),
    ],
    "MAE": [
        mean_absolute_error(y_test, y_pred_linear),
        mean_absolute_error(y_test, y_pred_huber),
    ]
}

# Создание таблицы с результатами
import pandas as pd
results_df = pd.DataFrame(metrics)

# Вывод результатов
print(results_df)

# Среднеквадратичная ошибка (MSE):
# У Linear Regression MSE чуть ниже (2900.19 против 2930.84),
# что означает, что эта модель лучше минимизирует квадраты ошибок.
# Однако MSE чувствителен к выбросам, поэтому при наличии выбросов
# Linear Regression может давать искусственно завышенные значения предсказаний.

# Средняя абсолютная ошибка (MAE):
# MAE у обеих моделей практически одинаковый (42.79 и 42.88). 
# Это показывает, что в среднем отклонения предсказаний от истинных
# значений схожи для обеих моделей.
# Однако Huber Regressor менее чувствителен к выбросам, поэтому его MAE 
# остаётся стабильным, даже если в данных есть аномалии.

               Model          MSE        MAE
0  Linear Regression  2900.193628  42.794095
1    Huber Regressor  2930.844631  42.879734
